# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [70]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vorgashor,67.5833,63.9500,-1.64,89,100,9.42,RU,1739044844
1,1,tual,-5.6667,132.7500,28.17,82,100,9.28,ID,1739044846
2,2,grytviken,-54.2811,-36.5092,2.49,70,53,5.04,GS,1739044847
3,3,cockburn town,21.4612,-71.1419,26.98,78,40,6.69,TC,1739044848
4,4,puerto natales,-51.7236,-72.4875,14.25,47,75,11.83,CL,1739044850


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [94]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_data_df.hvplot.points(x='Lng',y='Lat',s='Humidity',geo=True,tiles='OSM',color="City",scale=0.7,title='Cities and Humidity')

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df=city_data_df[(city_data_df['Max Temp']>21)&(city_data_df['Max Temp']<27)]
city_data_df=city_data_df[city_data_df['Cloudiness']==0]
city_data_df=city_data_df[city_data_df['Wind Speed']<4.5]
# Drop any rows with null values
# YOUR CODE HERE
city_data_df=city_data_df.dropna()
# Display sample data
# YOUR CODE HERE

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df=city_data_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name']=''

# Display sample data
# YOUR CODE HERE

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey':geoapify_key,
    'categories':'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = 'circle:'+str(row['Lng'])+","+str(row['Lat'])+","+str(10000)
    params["bias"] = 'proximity:'+str(row["Lng"])+','+str(row['Lat'])
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bahia de kino - nearest hotel: Hotel Hacienda
loreto - nearest hotel: No hotel found
afaahiti - nearest hotel: Omati Lodge
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
xai-xai - nearest hotel: No hotel found
fort pierce - nearest hotel: Dockside Inn
blenheim - nearest hotel: Hotel d'Urville
puri - nearest hotel: Divine Beach Homestay
hilo - nearest hotel: Dolphin Bay Hotel


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
47,47,bahia de kino,28.8333,-111.9333,22.46,41,0,3.29,MX,1739044842,Hotel Hacienda
292,292,loreto,22.2667,-101.9667,23.81,13,0,2.80,MX,1739045328,No hotel found
304,304,afaahiti,-17.7500,-149.2833,25.89,75,0,3.71,PF,1739045351,Omati Lodge
324,324,haiku-pauwela,20.9219,-156.3051,24.54,67,0,1.34,US,1739045384,Inn At Mama's Fish House
330,330,xai-xai,-25.0519,33.6442,26.10,91,0,3.07,MZ,1739045391,No hotel found
450,450,fort pierce,27.4467,-80.3256,26.12,71,0,3.13,US,1739045513,Dockside Inn
452,452,blenheim,-41.5160,173.9528,22.51,65,0,3.80,NZ,1739045590,Hotel d'Urville
507,507,puri,19.8000,85.8500,22.64,80,0,1.04,IN,1739045666,Divine Beach Homestay
514,514,hilo,19.7297,-155.0900,25.05,71,0,2.68,US,1739045677,Dolphin Bay Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [101]:
%%capture --no-display

# Configure the map plot
hotel_df.hvplot.points(x='Lng',y='Lat',geo=True,responsive=True,tiles='OSM',color="City",hover_cols=['Country','Humidity','Hotel Name'],title='Hotels with good weather')

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)